In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def detect_pose(image_path, protoFile, weightsFile, nPoints, POSE_PAIRS, threshold):
    # Load the image
    image = cv2.imread(image_path)

    # Load the Caffe model and perform a forward pass
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)
    blob = cv2.dnn.blobFromImage(image, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(blob)
    output = net.forward()

    # Get the dimensions of the input image
    height, width, _ = image.shape
    scaleX = width / output.shape[3]
    scaleY = height / output.shape[2]

    # Create an empty list to store detected keypoints
    points = []

    for i in range(nPoints):
        # Obtain the probability map
        probMap = output[0, i, :, :]

        # Find the global maximum of the probMap
        _, prob, _, point = cv2.minMaxLoc(probMap)

        # Scale the point to fit on the original image
        x = int(scaleX * point[0])
        y = int(scaleY * point[1])

        if prob > threshold:
            # Add the point to the list if the probability is greater than the threshold
            points.append((x, y))
        else:
            points.append(None)

    # Draw the skeleton on a copy of the image
    imSkeleton = image.copy()
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(imSkeleton, points[partA], points[partB], (255, 255, 0), 2)
            cv2.circle(imSkeleton, points[partA], 8, (255, 0, 0), thickness=-1, lineType=cv2.FILLED)

    # Display the image with the skeleton
    plt.imshow(imSkeleton[...,::-1])
    plt.show()

if __name__ == "__main__":
    # Define the paths to the Caffe model files
    protoFile = "model/caffe/pose_deploy_linevec_faster_4_stages.prototxt"
    weightsFile = "model/caffe/pose_iter_160000.caffemodel"

    # Number of keypoints and connections
    nPoints = 15
    POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13]]

    # Threshold for keypoint detection
    threshold = 0.1

    # Path to the input image
    image_path = "volleyball.jpg"

    # Detect and visualize the pose
    detect_pose(image_path, protoFile, weightsFile, nPoints, POSE_PAIRS, threshold)